In [2]:
import os
import cv2
import shutil
import PIL
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [38]:
BASE_DIR = '../'
MODEL_DIR = BASE_DIR+'model/'
IMAGES_DIR = BASE_DIR+'test/'
PREDICTIONS_DIR = BASE_DIR+'predictions/'
IMG_HEIGHT = 256
IMG_WIDTH = 256
CLASS_NAMES = ['correctly-masked', 'incorrectly-masked', 'not-masked']

In [3]:
model = keras.models.load_model(MODEL_DIR)

In [28]:
predict_dir = IMAGES_DIR
predict_ds = tf.keras.preprocessing.image_dataset_from_directory(
    predict_dir,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    seed = 123)
x_pred = []
for image, label in tqdm(predict_ds.unbatch().take(-1)):
    img = image.numpy().astype("uint8")
    x_pred.append(img)

x_pred = np.array(x_pred)

predictions = np.argmax(model.predict(x_pred), axis=-1)
predictions = predictions.reshape(1,-1)[0]

20it [00:00, 199.41it/s]

Found 532 files belonging to 1 classes.


532it [00:00, 1219.95it/s]


In [52]:
[Path(PREDICTIONS_DIR+clazz).mkdir(parents=True, exist_ok=True) for clazz in CLASS_NAMES]
for i in range(len(predictions)):
    img = Image.fromarray(x_pred[i])
    dest_dir = PREDICTIONS_DIR+CLASS_NAMES[predictions[i]] + '/'
    fname = dest_dir + str(i) + '.jpeg'
    img.save(fname)